In [1]:
import timm
import torch
import numpy as np
import torch.nn as nn
import os
from tqdm.auto import tqdm
from sklearn.metrics import roc_curve
from sklearn import metrics
import gc
import sys
import pandas as pd
import wandb
import numpy as np
import matplotlib.pyplot as plt
import timm
import numpy as np
import torch.nn as nn
import torch
from torchvision import datasets, models, transforms
from typing import Any
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn import metrics
import gc
import os 
import torch
import torch.nn as nn
import sys
import pandas as pd
import wandb

from einops import rearrange
from einops.layers.torch import Rearrange
import PIL.Image as Image
import PIL as pil
import time
import einops
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations.augmentations.transforms import ImageOnlyTransform
import random_word as word
import PIL.Image as Image
import einops
#Stage 1> A clean Swin transformer model will be trained trained
#Stage 2> The swin Transformer blocks are frozen and the new embeding layer is attached and trained
#The old embedding blocks are replaced by the CNNs.In this stage only the embeding layer is trained
#Stage 3> The entire model is unfrozen and trained

def metric(y_true, y_pred):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    auc = metrics.auc(fpr, tpr)
    return auc

def straightner(a):
    A = np.zeros((a[0].shape[0]*len(a)))
    start_index = 0
    end_index = 0
    for i in range(len(a)):
        start_index = i*a[0].shape[0]
        end_index = start_index+a[0].shape[0]
        A[start_index:end_index] = a[i]
    return A

def predictor(outputs):
    return np.argmax(outputs, axis = 1)


class restruct(torch.nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, img):
        """
        Args:
            img (Tensor): The stacked Image .
        Returns:
            Tensor: Restructured Image into 8 channels.
        """

        return einops.rearrange(torch.squeeze(img), 'h ( w c ) -> c h w ', w = 125, c=8)

def pil_loader(path: str) -> Image.Image:
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('L')

class Hybrid_embed(nn.Module):
    def __init__(self, feature_model, img_size, channels, efn_blocks, dims):
        super().__init__()


        self.feature_extractor = timm.create_model(feature_model,
                                                   features_only=True,
                                                   out_indices=[efn_blocks])


        self.feature_extractor.conv_stem = nn.Conv2d(channels,
                                       40,
                                       kernel_size=(3, 3),
                                       stride=(4, 4),
                                       padding=(1, 1),
                                       bias=False)

        with torch.no_grad():
                # NOTE Most reliable way of determining output dims is to run forward pass
                training = self.feature_extractor.training
                if training:
                    self.feature_extractor.eval()
                o = self.feature_extractor(torch.zeros(1, channels, img_size[0], img_size[1]))
                self.channel_output = o[0].shape[1]
                self.feature_extractor.train(training)

        self.embed_matcher = nn.Sequential(
            nn.Conv2d(self.channel_output, dims, kernel_size=(1, 1), stride=(1, 1), bias=False),
            nn.BatchNorm2d(dims, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.SiLU(inplace=True)
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        if isinstance(x, (list, tuple)):
            x = x[-1]  # last feature if backbone outputs list/tuple of features
        x = self.embed_matcher(x)
        x = x.flatten(2).transpose(1, 2)
        return x

class Hybrid_swin_effnet(nn.Module):
    def __init__(self, feature_model = "efficientnet_b3",img_size = (224,224), channels = 8, efn_blocks = 2, swin_blocks = 2, no_classes = 1):
        super().__init__()
        assert efn_blocks + swin_blocks == 4,f"The total no of blocks must be 4, instead {efn_blocks+swin_blocks} blocks provided "
        self.s1_flag = True
        self.s2_flag = True
        self.s3_flag = True
        self.swin_blocks = swin_blocks
        self.swin_backbone = timm.create_model("swin_tiny_patch4_window7_224",in_chans = 8)
        self.embeded_dim = self.swin_backbone.embed_dim * (2**(4 - self.swin_blocks))
        self.Hybrid_patch_embed = Hybrid_embed(feature_model = "efficientnet_b3",
                                                      img_size = (224,224),
                                                      channels = 8,
                                                      efn_blocks = 2,
                                                      dims = self.embeded_dim)
        self.swin_backbone.head = nn.Linear(self.swin_backbone.num_features, no_classes)


    def forward(self, image, stage):


        if stage == 2:
            #Attaching the new embeding layer

            self.swin_backbone.patch_embed = self.Hybrid_patch_embed

            for i in range((4- self.swin_blocks)):
                self.swin_backbone.layers[i] = nn.Identity()

            #Freezing the swin layers
            for layer in self.swin_backbone.layers:
                for para in layer.parameters():
                    para.requires_grad = False

           #Freezing the head
            for para in self.swin_backbone.head.parameters():
                    para.requires_grad = False

        if stage == 3:
            #Unfreezing the network
            for para in self.swin_backbone.parameters():
                para.requires_grad = True


        return self.swin_backbone(image).squeeze()




/global/u1/t/train130/Diptarko/work/lib64/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
NO_SAMPLE = 1024*1000

In [ ]:
torch.cuda.empty_cache()
model = Hybrid_swin_effnet()
model = model.to("cuda")

sample = torch.randn(1, 8, 224, 224, device = "cuda")
image = torch.randn(1024,8,224,224,device="cuda")
label = torch.randn(1024,device="cuda")
scaler = torch.cuda.amp.GradScaler()

with torch.no_grad():
    model(sample,1)
outputs_list = []
model.eval()
with torch.no_grad():
    for i in tqdm(range(1000)):
        outputs = model(image, 1)

gc.collect()

In [10]:
torch.cuda.empty_cache()
model = Hybrid_swin_effnet()
model = model.to("cuda")

sample = torch.randn(1, 8, 224, 224, device = "cuda")
image = torch.randn(1024,8,224,224,device="cuda")
label = torch.randn(1024,device="cuda")
scaler = torch.cuda.amp.GradScaler()


outputs_list = []
model.eval()
with torch.no_grad():
    for i in tqdm(range(1000)):
        outputs = model(image, 1)

gc.collect()



  9%|▉         | 90/1000 [00:57<09:39,  1.57it/s]


KeyboardInterrupt: 

In [7]:
import coat 
image_size = (224,224)
in_channels = 8
num_blocks = [2, 2, 3, 5, 2]
channels = [64, 96, 192, 384, 768]
num_classes = 1
model = coat.CoAtNet(image_size = image_size,
                        in_channels = in_channels,
                    num_blocks = num_blocks,
                    channels = channels,
                    num_classes = num_classes)

model = model.to("cuda")

sample = torch.randn(1, 8, 224, 224, device = "cuda")
image = torch.randn(1024,8,224,224,device="cuda")
label = torch.randn(1024,device="cuda")
scaler = torch.cuda.amp.GradScaler()


model.eval()
with torch.no_grad():
    for i in tqdm(range(1000)):
        outputs = model(image)

gc.collect()


100%|██████████| 1000/1000 [09:25<00:00,  1.77it/s]


24514

In [8]:
torch.cuda.empty_cache()
image_size = (128,128)
in_channels = 8
num_blocks = [2, 2, 3, 5, 2]
channels = [64, 96, 192, 384, 768]
num_classes = 1
model = coat.CoAtNet(image_size = image_size,
                        in_channels = in_channels,
                    num_blocks = num_blocks,
                    channels = channels,
                    num_classes = num_classes)

model = model.to("cuda")

sample = torch.randn(1, 8,128, 128, device = "cuda")
image = torch.randn(1024,8,128,128,device="cuda")
label = torch.randn(1024,device="cuda")
scaler = torch.cuda.amp.GradScaler()


model.eval()
with torch.no_grad():
    for i in tqdm(range(1000)):
        outputs = model(image)

gc.collect()

100%|██████████| 1000/1000 [03:00<00:00,  5.54it/s]


7007

In [20]:
torch.cuda.empty_cache()
model=timm.create_model("resnet18",in_chans=8)
model = model.to("cuda")

sample = torch.randn(1, 8, 224, 224, device = "cuda")
image = torch.randn(1024,8,224,224,device="cuda")
label = torch.randn(1024,device="cuda")
scaler = torch.cuda.amp.GradScaler()


outputs_list = []
model.eval()
with torch.no_grad():
    for i in tqdm(range(1000)):
        outputs = model(image)

gc.collect()


 36%|███▌      | 362/1000 [01:08<02:00,  5.29it/s]


KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()
model=timm.create_model("resnet18",in_chans=8)
model = model.to("cuda")

sample = torch.randn(1, 8, 125, 125, device = "cuda")
image = torch.randn(1024,8,125,125,device="cuda")
label = torch.randn(1024,device="cuda")
scaler = torch.cuda.amp.GradScaler()


outputs_list = []
model.eval()
with torch.no_grad():
    for i in tqdm(range(1000)):
        outputs = model(image)

gc.collect()

 46%|████▌     | 456/1000 [00:50<01:01,  8.91it/s]